# Navigating Networks

Many problems can be represented as networks.  The obvious examples are social networks, but maps and roads for directions are examples of these problems, and even fraud detection.  To understand networks, we represent the data with a graph.  Exploring these structures efficiently can be challenging, but several techniques have been developed to enable their use.

## Breadth-First Search

Methodology:

1. Represent graph path as a list - the list length equals the length of the path
2. Keep dictionary where keys are nodes and values are lists of paths
3. Maintain queue of unexplored nodes in pairs of as (new_node, node) to remember how to get to each one
4. First identification of new nodes (first time the node is taken out of the queue) means it is the shortest path has been found
5. Graph exploration is finished when no nodes are left in the queue, or at least the parts reachable from our starting node.

We will apply this to the example of users in hypothetical social network

In [1]:
users = [
    {'id': 0, 'name':'Hero'},
    {'id': 1, 'name':'Dunn'},
    {'id': 2, 'name':'Sue'},
    {'id': 3, 'name':'Chi'},
    {'id': 4, 'name':'Thor'},
    {'id': 5, 'name':'Clive'},
    {'id': 6, 'name':'Hicks'},
    {'id': 7, 'name':'Devin'},
    {'id': 8, 'name':'Kate'},
    {'id': 9, 'name':'Klein'}
]

friendships = [(0,1), (0,2), (1,2), (1,3), (2,3), (3,4),
              (4,5), (5,6), (5,7), (6,8), (7,8), (8,9)]

# give each user a friends list
for user in users:
    user["friends"] = []

# and populate it
for i, j in friendships:
    # this works because users[i] is the user whose id is i
    users[i]["friends"].append(users[j]) # add i as a friend of j
    users[j]["friends"].append(users[i]) # add j as a friend of i

In [12]:
from collections import deque
from functools import partial, reduce
import import_ipynb
from linear_algebra import *

importing Jupyter notebook from linear_algebra.ipynb


In [3]:
def shortest_paths_from(from_user):
    
    # a dictionary from 'user_id' to all shortest paths to that user
    shortest_paths_to = {from_user['id'] : [[]]}
    
    # a queue of (previous user, next user)
    frontier = deque((from_user, friend) for friend in from_user['friends'])
    
    while frontier:
        
        prev_user, user = frontier.popleft()     # remove first user in queue
        user_id = user['id']
        
        paths_to_prev_user = shortest_paths_to[prev_user['id']]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]
        
        old_paths_to_user = shortest_paths_to.get(user_id, [])
        
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')
            
        new_paths_to_user = [path
                            for path in new_paths_to_user
                            if len(path) <= min_path_length
                            and path not in old_paths_to_user]
        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user
        
        # add new neighbors to frontier
        frontier.extend((user, friend)
                       for friend in user['friends']
                       if friend['id'] not in shortest_paths_to)
        
        return shortest_paths_to

In [4]:
for user in users:
    user['shortest_paths'] = shortest_paths_from(user)

### Betweenness Centrality

Betweenness centrality can be thought of as a measure of the control a node has on the flow of information through a graph.  Essentially it measures the number of shortest paths in the graph through a node.

In [5]:
for user in users:
    user['betweenness_centrality'] = 0.0

for source in users:
    source_id = source['id']
    for target_id, paths in source['shortest_paths'].items():
        if source_id < target_id:
            num_paths = len(paths)
            contrib = 1./num_paths
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]['betweenness_centrality'] += contrib

### Closeness Centrality

The closeness centrality of a node is how close it is to other nodes based on the shortest paths to the rest.  While betweenness centrality measures how much information flows through a node, closeness centrality measures how well a node "brodcasts" information, i.e. how quickly each node can get information out to the rest of the graph.

In [6]:
def farness(user):
    """the sum of the lengths of the shortest paths to each other user"""
    return sum(len(paths[0])
               for paths in user["shortest_paths"].values())

for user in users:
    user["closeness_centrality"] = 1 / farness(user)

### Eigenvector Centrality

Eigencentraility takes into account how well connected the nodes are that a node is connected to and serves as a good general measure of connectedness.  This can be thought of as an overall influence on a network.

In [13]:
def matrix_product_entry(A, B, i, j):
    return dot(get_row(A, i), get_column(B, j))

def matrix_multiply(A, B):
    n1, k1 = shape(A)
    n2, k2 = shape(B)
    if k1 != n2:
        raise ArithmeticError("incompatible shapes!")

    return make_matrix(n1, k2, partial(matrix_product_entry, A, B))

def vector_as_matrix(v):
    """returns the vector v (represented as a list) as a n x 1 matrix"""
    return [[v_i] for v_i in v]

def vector_from_matrix(v_as_matrix):
    """returns the n x 1 matrix as a list of values"""
    return [row[0] for row in v_as_matrix]

def matrix_operate(A, v):
    v_as_matrix = vector_as_matrix(v)
    product = matrix_multiply(A, v_as_matrix)
    return vector_from_matrix(product)

def find_eigenvector(A, tolerance=0.00001):
    guess = [1 for __ in A]

    while True:
        result = matrix_operate(A, guess)
        length = magnitude(result)
        next_guess = scalar_multiply(1/length, result)

        if distance(guess, next_guess) < tolerance:
            return next_guess, length # eigenvector, eigenvalue

        guess = next_guess

In [14]:
def entry_fn(i, j):
    return 1 if (i, j) in friendships or (j, i) in friendships else 0

n = len(users)
adjacency_matrix = make_matrix(n, n, entry_fn)

eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

In [19]:
print("Eigenvector Centrality")
for user_id, centrality in enumerate(eigenvector_centralities):
    print(user_id, centrality)
print()

Eigenvector Centrality
0 0.38578006614957344
1 0.5147902322356226
2 0.5147902322356226
3 0.47331220396377677
4 0.23361029944966002
5 0.1501458150031844
6 0.08355561051056493
7 0.08355561051056493
8 0.07284034177922594
9 0.027294660139652423



### Directed Graphs

Directed graphs are a type of graph where there is a clear flow between nodes. 

In [15]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1), (1, 3),
                (2, 3), (3, 4), (5, 4), (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []       # add one list to track outgoing endorsements
    user["endorsed_by"] = []    # and another to track endorsements

for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])


In [16]:
endorsements_by_id = [(user["id"], len(user["endorsed_by"]))
                      for user in users]

sorted(endorsements_by_id,
       key=lambda pair: pair[1],
       reverse=True)

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [17]:
def page_rank(users, damping = 0.85, num_iters = 100):

    # initially distribute PageRank evenly
    num_users = len(users)
    pr = { user["id"] : 1 / num_users for user in users }

    # this is the small fraction of PageRank
    # that each node gets each iteration
    base_pr = (1 - damping) / num_users

    for __ in range(num_iters):
        next_pr = { user["id"] : base_pr for user in users }
        for user in users:
            # distribute PageRank to outgoing links
            links_pr = pr[user["id"]] * damping
            for endorsee in user["endorses"]:
                next_pr[endorsee["id"]] += links_pr / len(user["endorses"])

        pr = next_pr

    return pr

In [18]:
print("PageRank")
for user_id, pr in page_rank(users).items():
    print(user_id, pr)

PageRank
0 0.0404553415061296
1 0.044921190893169885
2 0.044921190893169885
3 0.0404553415061296
4 0.06785083675770529
5 0.04344422700587085
6 0.03346379647749512
7 0.03346379647749512
8 0.04344422700587085
9 0.03346379647749512
